In [6]:
import plotly.graph_objects as go
from scipy.stats import norm
import math
from datetime import datetime
import numpy as np

# Valuation

In a complete and arbitrage free capital market the market value of any asset (V) can be expressed by the sum of the present value of future cash flows using the risk-free rate ($r_f$) as the discount factor and the risk-neutral probability measure
$E^*$ (**Fundamental Asset Pricing Theorem – FAPT**):

$$V = \sum_{t=1}^{T} \frac{E^*(FCF_t)}{(1 + r_f)^t}$$

For company valuation, under the assumption that a constant stochastic discount factor exists,
this is equivalent to the **DCF entity** (Discounted Cash Flow) approach:

$$V = \sum_{t=1}^{T}  \frac{E(FCFF_t)}{(1 + WACC)^t}$$

where 
* $E()$ denotes the objective probability measure
* WACC denotes the firm’s cost of capital on a complete and arbitrage free capital market

## Entity vs Equity Approach

### Entity Approach

Free Cash Flow of the Firm (Weighted cost of capital (WACC)):

$$V_0 = \sum_{t=1}^{T} \frac{E(FCFF_t)}{(1 + WACC)^t}$$

### Equity Approach

Cash Flows of Debtholders (Cost of the firm's dept capital) and Free Cash Flows of Equityholders (Cost of equity):

$$V_0 = \sum_{t=1}^{T} \frac{E(FCFE_t)}{(1 + r_E)^t}$$

* Value of dept is often approximated by the book value of dept
* value of equity = # shares multiplied by the share price

### Calculating free cash flow to equity (FCFE)

Net Income = $EBT - Taxes$
* EBT: earnings before taxes

Net Income + Depreciation - Investments - NOWC - Cash Flow to Debt = FCFE
* NOWC = receivables + inventory - payables - operational provisions
* Cash Flow to Debt = repayment of debt - new debt issued

=> the amount of money that can be distributed to equity holders

### Calculating free cash flow to the firm (FCFF)

Net Operating Profit After Taxes (NOPAT) = Earnigns before interest and taxes (EBIT) - Taxes
* Taxes = Tax rate * EBT

NOPAT + Depreciation - Investments - NOWC = FCFF

=> the amount of money that can be distributed to all suppliers of capital

## Entity Value (Firm Value) vs. Enterprise Value

Entity Value (Firm Value) = Enterprise Value (=Value of Operations) + Value of Non-Operating Assets

$$V_{Op} = \sum_{t=1}^{\infty} \frac{E(FCFF_t)}{(1 + WACC)^t}$$

* Firm value or entity values are used interchangeably; they indicate the value of all operating and non-operating assets, i.e. $V_{Op} + V_{NOA} = V_D + V_E$
* Enterprise value reflects the value of operating assets only, i.e. $V_{Op} = V_E + (V_D - V_{NOA}) = V_E + NetDebt$
* Debt reduced by the market value of non-operating assets (mostly cash) is called net debt, i.e  $NetDebt = V_D - V_{NOA}$. **Therefore, when calculating the DCF enterprise value the capital structure has to be measured on the basis of the net debt.**

Value of non-operating assets:
* Marketable securities
* Ownership of non-controlling interest in another company







## Cash Flow Projection as a two step procedure

- Phase I: Detailed planning
- Phase II: Constant growth

Calculating the terminal value $(TV)$:

$$TV_{T_g} = \frac{FCFF_{T_g+1}}{WACC - g}$$

$$TV_{T_0} = TV_{T_g} * \frac{1}{(1 + WACC)^{T_g-T_0}}$$

$$FCFF_{T_g+1} = FCFF_{T_g} * (1 + g)$$

Comment: If you have projected CFs for a given period ending in "t" and you use the Gordon 
Growth Formula to calculate the TV after that period (FCFF(t+1)/ cost of capital - growth rate) 
you get a terminal value with respect to "t" and not "t+1". In this example the perpetuity starts 
in t=4 and when we calculate its value we get a result that is relative to t=3. This is why we then 
have to discount it by 3 and not 4 periods to receive the PV.

Comment: the period $t$ doesn't need to be an integer! If the projections don't start at 1st 
january od the first year, we need to take that in account

### Issues in cash flow projection

#### Detailed forecast of next three years CFs with three alternative methods

* Historical estimates
  * Sensitive to choice of method and time period
  * Possibly no good proxy for future (esp. if CFs negative)
* Expert estimates: Analysts forecasts
  * Better for larger, better covered firms
  * Not necessarily accurate, possibly positively biased
* Fundamental business modeling
  * Delivers most accurate, subjective value estimate
  *Takes time, skill and insight

#### Terminal value (TV) determined by long-term future CF and long-term growth

* Growth rate g assumption essential
  * Usually, inflation plus long-term growth of economy
* Different terminal value methods possible
  * Perpetuity growth model: TV = CF / (k – g)
  * Exit multiple approach

## Estimation of the cost of equity

### Cost of equity estimation is (mostly) based on CAPM

**Rational investors tend to diversify their risks** thus only the market risk component is compensated by the market.

CAPM pricing formula
$$r_E = r_{riskfree} + \beta_{stock} \cdot (r_{market\ return} - r_{riskfree})$$

**From the CAPM pricing formula it follows that** securities are not priced with respect to their stand-alone risk but their with respect to their market risk only!

### Issues in implementing this model

* Market risk premiums are highly dispututied
* IDW currently recommends 6 to 8% (pre-tax)
* Beta estimation is not very robust and subject to discretion
  * Sampling frequency (daily, monthly, etc.)
  * Estimation window (250 days, 60 months, etc.)
  * Market geography (national, world-wide)
  * Market index
  * Alternative market models (CAPM, FF3FM, etc.)
  * Individual vs. industry betas
  * Problem of de- and releveraging

## How to de-/re-leverage a beta

* Betas of different companies need to be re-leveraged to make them comparable
  * Because company-specific leverage has large influence
  * Beta with leverage of target company has to be calculated
  * Alternatively, adjustment can be done via the WACC-formula
* For firms with constant risk-free debt de-leveraged equity (asset) beta can be calculated according to the Hamada-Equation

$$\beta_E = \beta_U \left(1 + \frac{V_D}{V_E}(1-T) \right)$$

## Derivation of delevered (or unlevered) beta:

## WACC

If a firm is financed by debt and equity simultaneously, then the discount rate in the entity approach must reflect this fact

➔ weighted average cost of capital (WACC)

$$WACC = r_D (1-T) \frac{V_D}{V} + r_E \frac{V_E}{V}$$

where
* r_D is the cost of debt,
* T is the corporate tax rate,
* r_E is the cost of equity,
* V_D the market value of outstanding net debt,
* V_E the market value of outstanding equity

Note that V=V_D+V_E holds by definition

* Problem 1: circularity problem
  ➔ WACC provides a tool to determine V and V_E but both parameters are needed as inputs to determine WACC
  ➔ may use rollback methods to solve the problem

* Problem 2: Debt regime
  ➔ The left hand side WACC assumes a constant leverage (based on market values)

* Alternative method: APV Approach
  ➔ APV splits the entity value of a debt finance firm into the value as if it would be fully equity financed and the tax shield of debt
  ➔ may use rollback methods to solve the problem

## Risk-free rate and term structure

### Estimating the term structure
- Ideally the term structure should be reflected in the discount rate
  - This can be done by looking at the empirical term structure of forward rates
  - Alternatively, the Svensson-method can be applied

$$i_t^p \cdot 100 = \beta_0 + \beta_1\frac{1-\exp\left(-\frac{t}{\tau_1}\right)}{\frac{t}{\tau_1}} + \beta_2\left(\frac{1-\exp\left(-\frac{t}{\tau_1}\right)}{\frac{t}{\tau_1}} - \exp\left(-\frac{t}{\tau_1}\right)\right) + \beta_3\left(\frac{1-\exp\left(-\frac{t}{\tau_2}\right)}{\frac{t}{\tau_2}} - \exp\left(-\frac{t}{\tau_2}\right)\right)$$

- This gives the continuously compounded risk-free rate for maturity t; all parameters are daily reported by the Bundesbank or the FED

### Practitioners, however, often operate with a single discount rate
- This discount rate k is derived by solving (numerically, t≈250) the equation

$$\sum_{t=1}^{\infty} \frac{(1+g)^t}{(1+k_t)^t} = \frac{1+g}{k-g}$$

### Problems with Svensson-method
* Note, however, that applying the Svensson-Method beyond the observable 30-year maturity can lead to strange patterns in the term structure:
* One solution is to estimate an ultimate forward rate (UFR).
* Alternatively, it is often assumed that beyond the year 30 the term structure is flat.

## Valuation and personal taxes

- Hitherto we have looked at corporate taxes only, personal taxes were ignored

### Taking into account personal taxes
- According to IDW S1, cif. 28, personal taxes on any distribution made by the company has to be taken into account (after tax valuation).
- The after tax cost of equity is calculated according to the tax CAPM. Under the current tax regime (Abgeltungssteuer) it follows:

$$r_E^{AT} = (1-\pi)r_{riskfree} + \beta_{stock} \cdot (r_{market\ return}^{AT} - (1-\pi)r_{riskfree})$$

where π is the personal tax rate (26.4%) and AT stands for after tax

- According to IDW the after tax market risk premium should be around 5 to 6%

In [9]:
# CAPM
def calculate_re(rf, beta, rm):
    '''
    Calculates re, the cost of an equity
    Parameters
    ----------
    rf: float
    risk-free rate
    beta: float
    beta of the stock (market risk)
    rm: float
    market return
    Notes
    -----
    rm-rf is the premium of the market over the risk-free rate
    '''
    re = rf + beta * (rm - rf)
    return re


# Hamada-Equation, supposing beta_debt=0
def hamada(beta_u, Vd, Ve, tau):
    # beta_u: unlevered beta
    # Vd: debt
    # Ve: equity
    # tau: tax rate
    return beta_u * (1 + Vd / Ve * (1 - tau))


def hamada_(beta_e, Vd, Ve, tau):
    # beta_e: levered beta
    # Vd: debt
    # Ve: equity
    # tau: tax rate

    return beta_e * (1 / (1 + Vd / Ve * (1 - tau)))


def calculate_wacc(ke, kd, Ve, Vd, tau):
    V = Ve + Vd
    return ke * Ve / V + kd * (1 - tau) * Vd / V



In [2]:
hamada(.8, 9, 6, .3)

1.64

In [10]:
# 1. Calculate the unlevered beta
Vd_comp = 0.2
Ve_comp = 7.1

# debt_equity_ratio = 0.8/2.3
# 
# if not Ve_comp:
#     Ve_comp = Vd_comp/debt_equity_ratio
# else:
#     Vd_comp = Ve_comp*debt_equity_ratio

tau_comp = .33
beta_e_comp = 1.24
beta_u_comp = hamada_(beta_e_comp, Vd_comp, Ve_comp, tau_comp)
print(f'The unlevered beta is {beta_u_comp:.2f}')

# We assume beta_u_comp = beta_u
beta_u = beta_u_comp

# 2. Calculate the relevered beta
Vd = 400
Ve = None

debt_equity_ratio = 0.8 / 2.3

if not Ve:
    Ve = Vd / debt_equity_ratio
else:
    Vd = Ve * debt_equity_ratio

tau = .33
beta = hamada(beta_u, Vd, Ve, tau)

print(f'The relevered beta is {beta:.2f}')

# 3. Calculate the cost of equity using the relevered beta and CAPM
# cost of equity
rm = .121
rf = .076
re = calculate_re(rf, beta, rm)

print(f'The cost of equity is {re * 100:.2f}%')

# 4. Calculate the cost of debt
ke = re
kd = .1
Ve = Ve
Vd = 400
WACC = calculate_wacc(ke, kd, Ve, Vd, tau)

print(f'The WACC is {WACC * 100:.2f}%')

The unlevered beta is 1.22
The relevered beta is 1.50
The cost of equity is 14.35%
The WACC is 12.38%


## Present Value (PV)

In [46]:
def calculate_PV(cashflows, rate):
    '''
    Calculates the Present Value of a company based on the projection of the cashflow
    cashflows: list[float]
    rate: float
    constant rate utilized, can be WACC
    '''
    PV = sum([cf / ((1 + rate) ** (i + 1)) for i, cf in enumerate(cashflows)])
    return PV

def calculate_PV_perpetuity(cashflow_t, rate, g=None, t=None, discount_t=None):
    '''
    Calculates the Present Value of a perpetuity based on the projection of the cashflow
    cashflow_t: float
    cashflow at time t
    rate: float
    constant rate utilized, can be WACC or cost of equity (k_u)
    g: float
    growth rate
    t: int
    number of periods
    '''
    
    if t is None:
        if g is None:
            PV = cashflow_t / rate
        else:
            PV = cashflow_t / (rate - g)
    else:
        if g is None:
            PV = cashflow_t / ((1 + rate) ** t)
        else:
            PV = cashflow_t * (1 + g) / (rate - g) / ((1 + rate) ** t)

    if discount_t is not None:
        PV = PV / ((1 + rate) ** discount_t)
    return PV


def calculate_PV_interest_tax_shields(interest_expenses, rate, tau):
    '''
    Calculates the Present Value of the interest tax shields
    interest_expenses: list[float]
    rate: float
    constant rate utilized, can be WACC or cost of debt (k_d)
    tau: float
    tax rate (0 < tau < 1)
    '''
    PV = sum([interest * tau / ((1 + rate) ** (i + 1)) for i, interest in enumerate(interest_expenses)])
    return PV

def calculate_PV_tax_shields_perpetuity(FCFF, g, WACC, n, rate, dept_rate):
    '''
    Calculates the Present Value of the tax shields beyond a certain year in perpetuity
    FCFF: float
        Free Cash Flow to the Firm at the final projected year
    g: float
        Growth rate of the cash flows
    WACC: float
        Weighted Average Cost of Capital
    n: int
        Number of years till the final projected year
    rate: float
        constant rate utilized, can be cost of equity
    '''
    TVL = (FCFF * (1 + g)) / (WACC - g)
    TVT = (FCFF * (1 + g)) / (rate - g)
    PV_TS = (TVL - TVT) / ((1 + dept_rate) ** n)
    return PV_TS

def calculate_firm_value(V_U, PV_TS):
    '''
    Calculates the total value of the levered firm
    V_U: float
        Value of the unlevered firm
    PV_TS: float
        Present Value of the Tax Shields
    '''
    V_L = V_U + PV_TS
    return V_L

def calculate_equity_value(V_L, V_D, shares_outstanding):
    '''
    Calculates the equity value and value per share
    V_L: float
        Value of the levered firm
    V_D: float
        Value of the firm's debt
    shares_outstanding: float
        Number of shares outstanding
    '''
    V_E = V_L - V_D
    value_per_share = V_E / shares_outstanding
    return V_E, value_per_share

In [48]:
cashflows = [5404, 4311, 2173, 2336, 2536]
rate = 0.14  # from last code-cell (WACC)
PV1 = calculate_PV(cashflows, rate)
print(f"The present value of the company is {PV1:.2f}")


The present value of the company is 12224.46


In [49]:
cashflow = 2536
rate = 0.14  # from last code-cell (WACC)
g = 0.03
t = 5
PV2 = calculate_PV_perpetuity(cashflow, rate, g, t)
print(f"The present value of the company after date is {PV2:.2f}") # DOnt forget to add the PV of the cashflows before the terminal value

The present value of the company after date is 12333.02


In [50]:
interest_expenses = [3384, 3004, 3111, 3294, 3483]
rate = 0.135
tau = 0.34
PV3 = calculate_PV_interest_tax_shields(interest_expenses, rate, tau)
print(f"The present value of the interest tax shields is {PV3:.2f}")

The present value of the interest tax shields is 3833.56


In [51]:
FCFF = 2536
g = 0.03
WACC = 0.128
n = 5
rate = 0.14
dept_rate = 0.135
PV4 = calculate_PV_tax_shields_perpetuity(FCFF, g, WACC, n, rate, dept_rate)
print(f"The present value of the tax shields in perpetuity is {PV4:.2f}")

The present value of the tax shields in perpetuity is 1543.72


In [52]:
VL = PV1 + PV2 + PV3 + PV4
outstanding_dept = 5000
VE = VL - outstanding_dept
shares_outstanding = 229
Value_per_share = VE / shares_outstanding

print(f"The total value of the levered firm is {VL:.2f}")
print(f"The equity value is {VE:.2f}")
print(f"The value per share is {Value_per_share:.2f}")

The total value of the levered firm is 29934.76
The equity value is 24934.76
The value per share is 108.89


In [22]:
def get_entity_value_dcf(cashflows, rate, g, pv_debt=None, valuation_date=None):
    """Calculate the entity value using the DCF method."""
    # Terminal value with perpetual growth
    terminal_value = cashflows[-1] * (1 + g) / (rate - g)
    
    # Present value of terminal value
    if valuation_date:
        days_in_year = 365
        days_to_valuation = (datetime.strptime(valuation_date, '%Y-%m-%d') - datetime.strptime(valuation_date.split('-')[0] + '-12-31', '%Y-%m-%d')).days
        days_fraction = days_to_valuation / days_in_year
        PV_final = terminal_value / (1 + rate) ** (len(cashflows) - 1 + days_fraction)
    else:
        PV_final = terminal_value / (1 + rate) ** len(cashflows)
    
    # Present value of cash flows
    PV = calculate_PV(cashflows, rate)
    
    # Entity value calculation
    if pv_debt:
        V = PV + PV_final - pv_debt
    else:
        V = PV + PV_final

    return {
        'terminal_value': terminal_value,
        'PV_final': PV_final,
        'PV': PV,
        'V': V
    }

In [21]:
cashflows = [5404, 4311, 2173, 2336, 2536]
# g - growth rate
g = .04
# re - cost of equity
re = .1435
# rd - cost of debt
rd = .1
# Vd - debt
Vd = 400

debt_equity_ratio = 0.8 / 2.3
# Ve - equity
Ve = None

if not Ve:
    Ve = Vd / debt_equity_ratio
else:
    Vd = Ve * debt_equity_ratio

tau = .33

WACC = calculate_wacc(re, rd, Ve, Vd, tau)

rate = WACC
result = get_entity_value_dcf(cashflows, rate, g, Vd)

print(f"The Enterprise Value (Terminal Value) is {result['terminal_value']:.2f}")
print(f"The Present Value of the Terminal Value is {result['PV_final']:.2f}")
print(f"The Present Value of the company (equity) is {result['V']:.2f}")



The Enterprise Value (Terminal Value) is 31488.79
The Present Value of the Terminal Value is 17570.81
The Present Value of the company (equity) is 29804.57


In [24]:
# if only sales and ebit are given as well as depr charches and we have to calculate the FCFF first using the following

# (in million EUR)

sales = [9.40, 10.25, 11.79, 13.44, 15.05, 16.86, 18.88, 20.77, 22.84, 25.13]
ebit = [2.77, 3.23, 3.54, 4.03, 4.21, 4.72, 5.29, 5.81, 6.4, 7.04]
depreciation_percentage = 0.15
capex_percentage = 0.18
working_capital_percentage = 0.10
change_in_nwc_initial = 0.5
tax_rate = 0.20
g = 0.02
wacc = 0.1093
num_shares = 1189890
market_value_debt = 2.6

# Calculate FCFF for each year
fcff = []
for i in range(len(sales)):
    depreciation = sales[i] * depreciation_percentage
    capex = sales[i] * capex_percentage
    nwc = sales[i] * working_capital_percentage
    if i == 0:
        change_in_nwc = change_in_nwc_initial
    else:
        change_in_nwc = nwc - (sales[i - 1] * working_capital_percentage)
    nopat = ebit[i] * (1 - tax_rate)
    fcff.append(nopat + depreciation - capex - change_in_nwc)

# Set the valuation date
valuation_date = '2015-04-08'

# Calculate the entity value using DCF
result = get_entity_value_dcf(fcff, wacc, g, market_value_debt, valuation_date)

# Calculate the share price
enterprise_value = result['V']
equity_value = enterprise_value - market_value_debt
share_price = equity_value / (num_shares / 1000000)

print(f"The Enterprise Value (Terminal Value) is {result['terminal_value']:.2f} million EUR")
print(f"The Present Value of the Terminal Value is {result['PV_final']:.2f} million EUR")
print(f"The Present Value of the company (equity) is {result['V']:.2f} million EUR")
print(f"The Share Price is {share_price:.4f} EUR")

The Enterprise Value (Terminal Value) is 53.10 million EUR
The Present Value of the Terminal Value is 22.52 million EUR
The Present Value of the company (equity) is 36.41 million EUR
The Share Price is 28.4145 EUR


## Specific APV cases

## Enterprise Value (WACC):

$$V_0 = \frac{E[FCFF_1]}{(1+WACC)^1} + \frac{E[FCFF_2]}{(1+WACC)^2} + \frac{E[FCFF_3]}{(1+WACC)^3} + ...$$

## Enterprise Value (APV):

$$V_0 = \frac{E[FCFF_1]}{(1+r_U)^1} + \frac{E[FCFF_2]}{(1+r_U)^2} + \frac{E[FCFF_3]}{(1+r_U)^3} + ... + PVTS$$

The Adjusted Present Value Method (APV) is a general approach for valuing firms that do not have constant debt ratios. The enterprise value is split-up into a value of the unlevered firm V_U and present value of the tax shield PVTS. Note, however, that deriving the unlevered cost of capital r_U in general is not obvious, as for an investor holding all the outstanding claims of a firm the following relationship applies (r_T is the expected return associated with the tax shield):

$$V_L = V_E + V_D = V_U + PVTS$$

$$r_EV_E + r_DV_D = r_UV_U + r_TPVTS$$

## The APV-method leads to simple solutions only in specific cases

### Assume a constant leverage ratio (case I)
In this case r_T=r_U, as debt is proportional to firm value and tax shields. Therefore, debt has the same risk as free cash flows.

$$PVTS = \sum_{t=1}^{\infty} \frac{r_DV_{D,t}T}{(1+r_U)^t}$$
$$\quad r_U = r_E\frac{V_E}{V} + r_D\frac{V_D}{V}$$
$$\quad WACC = r_U - r_DT\frac{V_D}{V};$$

### Assume a constant debt level (case II)
In this case r_T=r_D, as debt is constant and the tax shield has the same risk as debt.

$$PVTS = \sum_{t=1}^{\infty} \frac{r_DV_DT}{(1+r_D)^t}$$
$$\quad r_U = r_E\frac{V_E}{V_E + V_D(1-T)} + r_D\frac{V_D(1-T)}{V_E + V_D(1-T)}$$

Combining this result with the well-know WACC-formula yields: $WACC = r_U \left(1 - T\frac{V_D}{V_E + V_D}\right)$

### These are two special solutions to the following general relationship

$$WACC = r_U - \frac{V_D}{V_E + V_D}T[r_D + k(r_U - r_D)]$$

where k measures the permanence of the debt level, i.e. k=0 in case I and k=1 in case II

In [13]:
def calculate_apv(re, rd, t, rf, rm, k, Vd=None, Ve=None):
    '''
    k=0: case I (constant leverage ratio) => rt = ru
    k=1: case II (constant debt level) => rt = rd
    '''
    if Vd:
        Ve = 1 - Vd
    else:
        Vd = 1 - Ve

    ru = rd * Vd + re * Ve
    WACC = ru - Vd * t * (rd + k * (ru - rd))
    betau = (ru - rf) / rm
    betae = (re - rf) / rm
    betad = (betau - Ve * betae) / Vd

    return {
        'ru': ru,
        'WACC': WACC,
        'betau': betau,
        'betae': betae,
        'betad': betad
    }


In [14]:
# re - cost of equity
re = 0.14
# rd - cost of debt
rd = 0.135
# t - tax rate
t = 0.34
# rf - risk-free rate
rf = 0.02
# rm - market return
rm = 0.03
# k - case (0=constant leverage ratio, 1=constant debt level)
k = 1  # 

result = calculate_apv(re, rd, t, rf, rm, k, Vd=.5)
print(f"The unlevered cost of equity is {result['ru']:.2f}")
print(f"The WACC is {result['WACC'] * 100:.2f}%")
print(f"The unlevered beta is {result['betau']:.2f}")
print(f"The levered beta is {result['betae']:.2f}")
print(f"The beta of the debt is {result['betad']:.2f}")


The unlevered cost of equity is 0.14
The WACC is 11.41%
The unlevered beta is 3.92
The levered beta is 4.00
The beta of the debt is 3.83


## Multiple Valuation


### Multiples as an outcome of the DCF-approach

If two firms have the same WACC and growth rate, the ratio of the enterprise value to the FCFF is the same for both. This ratio can be labelled as a multiple.

$$EV = FCFF_1 \times M$$

$$EV = \sum_{t=1}^{\infty} FCFF_t \cdot (1+g)^t \cdot (1+WACC)^{-t} = \frac{FFCF_1}{WACC - g}$$

$$\Rightarrow M = \frac{1}{WACC - g}$$

For a non-growing firm, $FCFF=EBIT(1-T)$ holds; hence, for firms with the same WACC and growth rate the ratio EV/EBIT should be the same.

Similarly, For firms with equal risk, growth rates and capital structure the cost of equity is the same; hence, they should have the same price/earnings ratio.

**EV / Sales**

Advantages:
* Applicable to young companies with negative Annual Net Profit
* Good availability of data

Disadvantages:
* Sales is a bad indicator of profit situation
* Problems with accounting effects (e.g. non-cash sales)
* EV reflects the tax shield of comparables → multiple relates pre-tax earnings figure to after-tax market value

**EV / EBITDA**

Advantages:
* Reflects operating profitability
* Influence of accounting because of depreciation eliminated

Disadvantages:
* Problems due to differences in CAPEX and change in NWC
* EV reflects the tax shield of comparables → multiple relates pre-tax earnings figure to after-tax market value

**EV / EBIT**

Advantages: 
* Reflects operating profitability
* Independent from capital structure decision → high international comparability

Disadvantages:
* Influence of depreciation policy
* EV reflects the tax shield of comparables → multiple relates pre-tax earnings figure to after-tax market value

**P / E**

Advantages: 
* Easy to communicate
* High degree of aggregation
* Suitable if capital structure is similar

Disadvantages:
* Annual Net Profit distorted due to depreciation, taxes, capital structure

### Enterprise vs. Equity multiples

Value of firm = Value of equity (Often approximated by the book value of dept) + Value of net debt (market value of equity = # shares * share price)

* Enterprise multiples based on measures of overall performance (e.g. EBIT, EBITDA, Sales, Customers)
* Share price multiples based on measures of equity return (e.g. Price per share / Earnings (P/E-multiple))

### Finding peer group firms is the fundamental problem in multiple valuation

**Trading multiples**
* Multiples are derived from share prices of companies listed on stock exchange
* "Peer Group" usually based on firms of same industry (same size, etc)
* Implies "correct" valuation of comparable companies by the market
* For valuation of acquisitions: share prices usually do not contain a control premium; thus it has to be taken into account separately as a premium

**Transaction multiples**
* Multiples are derived from observed acquisitions prices of recent comparable transactions / from observed IPOs from recent IPOs
* Acquisition prices usually contain control or strategic premia: thus no need to be taken into account separately!

**Problems**
* It is often hard to find comparable firms.
* The average ratio for the sample of comparable firms often has a wide range.
    * For example, the average P/E ratio might be 20, but the range could be from 10 to 50. How do you know whether your firm should be compared to the low, average, or high performers?

### Adjusting multiples is another important problem in multiple valuation
Various key figures can be used for calculating multiples (e.g.: Sales Revenue, EBITDA, EBIT, NOPLAT, Net Profit,...)

$$↓$$

Unique events and discretionary accounting policy may distort key data and thus company values calculated

$$↓$$

Key figures should be standardized and/or adjusted.


Adjustments
* Extraordinary expenses/earnings
* Disposition-contingent expenses/earnings (e.g. stock options, R&D expenses, non cash -revenues, pension reserves)

In [30]:
def calculate_multiples(price, revenue, earnings, ebit, ebitda, book_value, debt, num_shares):
    multiples = {
        'P/Rev': price / revenue * num_shares,
        'P/E': price / earnings * num_shares,
        'P/B': price / book_value * num_shares,
        'EV/EBIT': (price * num_shares + debt) / ebit,
        'EV/EBITDA': (price * num_shares + debt) / ebitda
    }
    return multiples


def calculate_mean_min_max_multiples(multiples_list):
    mean_multiples = {}
    min_multiples = {}
    max_multiples = {}
    
    for multiples in multiples_list:
        for key, value in multiples.items():
            if key not in mean_multiples:
                mean_multiples[key] = value
                min_multiples[key] = value
                max_multiples[key] = value
            else:
                mean_multiples[key] += value
                min_multiples[key] = min(min_multiples[key], value)
                max_multiples[key] = max(max_multiples[key], value)
                
    for key in mean_multiples:
        mean_multiples[key] /= len(multiples_list)
        
    return mean_multiples, min_multiples, max_multiples


def calculate_estimated_prices_at_multiples(multiples, price, revenue, earnings, ebit, ebitda, book_value, debt, num_shares):
    estimated_prices = {
        'P/Rev': max(0, multiples['P/Rev'] * revenue / num_shares),
        'P/E': max(0, multiples['P/E'] * earnings / num_shares),
        'P/B': multiples['P/B'] * book_value / num_shares,
        'EV/EBIT': (multiples['EV/EBIT'] * ebit - debt) / num_shares,
        'EV/EBITDA': (multiples['EV/EBITDA'] * ebitda - debt) / num_shares
    }
    return estimated_prices


def calculate_estimated_prices_range(mean_multiples, min_multiples, max_multiples, target_data):
    mean_prices = calculate_estimated_prices_at_multiples(mean_multiples, **target_data)
    min_prices = calculate_estimated_prices_at_multiples(min_multiples, **target_data)
    max_prices = calculate_estimated_prices_at_multiples(max_multiples, **target_data)
    
    return mean_prices, min_prices, max_prices


def estimate_stock_price_from_range(estimated_prices):
    mean_price = sum(estimated_prices[0].values()) / len(estimated_prices[0])
    min_price = sum(estimated_prices[1].values()) / len(estimated_prices[1])
    max_price = sum(estimated_prices[2].values()) / len(estimated_prices[2])
    
    return min_price, mean_price, max_price


def print_multiples(multiples, label):
    print(f"\n{label} Multiples:")
    for key, value in multiples.items():
        print(f"{key}: {value:.2f}")


def print_estimated_prices(estimated_prices, label):
    print(f"\n{label} Prices for Target Company:")
    for key, value in estimated_prices.items():
        print(f"{key}: {value:.2f}")


def print_estimated_stock_price_range(min_price, mean_price, max_price):
    print('\n' + '=' * 50)
    print(f"Estimated Stock Price for Target Company:")
    print(f"Min: {min_price:.2f}, Mean: {mean_price:.2f}, Max: {max_price:.2f}")
    print('=' * 50)

In [31]:
companies_data = {
    'company1': {
        'price': 60.91,
        'revenue': 129.872,
        'earnings': 9.272,
        'ebit': 9.346,
        'ebitda': 13.04,
        'book_value': 43.665,
        'debt': 86.689,
        'num_shares': 1.0698
    },
    'company2': {
        'price': 70.09,
        'revenue': 80.401,
        'earnings': 6.368,
        'ebit': 8.99,
        'ebitda': 12.092,
        'book_value': 37.22,
        'debt': 77.506,
        'num_shares': 0.6565
    },
    'target_company': {
        'price': 95.2,
        'revenue': 202.458,
        'earnings': 8.896,
        'ebit': 11.921,
        'ebitda': 20.505,
        'book_value': 89.991,
        'debt': 108.647,
        'num_shares': 0.5013
    }
}

# Calculate multiples for company1 and company2
company1_multiples = calculate_multiples(**companies_data['company1'])
company2_multiples = calculate_multiples(**companies_data['company2'])

# Print multiples for company1 and company2
print_multiples(company1_multiples, "Company 1")
print_multiples(company2_multiples, "Company 2")

# Calculate mean, min, and max multiples
mean_multiples, min_multiples, max_multiples = calculate_mean_min_max_multiples([company1_multiples, company2_multiples])

# Print mean, min, and max multiples
print_multiples(mean_multiples, "Mean")
print_multiples(min_multiples, "Min")
print_multiples(max_multiples, "Max")

# Estimate stock prices for target company
mean_prices, min_prices, max_prices = calculate_estimated_prices_range(mean_multiples, min_multiples, max_multiples, companies_data['target_company'])

# Print estimated prices for target company
print_estimated_prices(mean_prices, "Mean")
print_estimated_prices(min_prices, "Min")
print_estimated_prices(max_prices, "Max")

# Estimate stock price range from multiples
min_price, mean_price, max_price = estimate_stock_price_from_range((mean_prices, min_prices, max_prices))

# Print estimated stock price range
print_estimated_stock_price_range(min_price, mean_price, max_price)


Company 1 Multiples:
P/Rev: 0.50
P/E: 7.03
P/B: 1.49
EV/EBIT: 16.25
EV/EBITDA: 11.64

Company 2 Multiples:
P/Rev: 0.57
P/E: 7.23
P/B: 1.24
EV/EBIT: 13.74
EV/EBITDA: 10.22

Mean Multiples:
P/Rev: 0.54
P/E: 7.13
P/B: 1.36
EV/EBIT: 14.99
EV/EBITDA: 10.93

Min Multiples:
P/Rev: 0.50
P/E: 7.03
P/B: 1.24
EV/EBIT: 13.74
EV/EBITDA: 10.22

Max Multiples:
P/Rev: 0.57
P/E: 7.23
P/B: 1.49
EV/EBIT: 16.25
EV/EBITDA: 11.64

Mean Prices for Target Company:
P/Rev: 216.88
P/E: 126.47
P/B: 244.91
EV/EBIT: 139.82
EV/EBITDA: 230.35

Min Prices for Target Company:
P/Rev: 202.63
P/E: 124.71
P/B: 221.93
EV/EBIT: 110.00
EV/EBITDA: 201.10

Max Prices for Target Company:
P/Rev: 231.14
P/E: 128.23
P/B: 267.89
EV/EBIT: 169.64
EV/EBITDA: 259.59

Estimated Stock Price for Target Company:
Min: 172.08, Mean: 191.69, Max: 211.30


In [15]:
def calculate_wacc(tau, re, rd, Ve, Vd):
    V = Ve + Vd
    WACC = (1 - tau) * rd * Vd / V + re * Ve / V
    return WACC

In [16]:
tau = .3
re = .1
rd = .05
Vd = 3
Ve = 5
calculate_wacc(tau, re, rd, Ve, Vd)

0.075625

In [ ]:
def black_scholes_valuation(asset_value, debt_value, risk_free_rate, volatility, time_to_maturity):
    """
    Calculate market value of equity and debt using the Black-Scholes model.
    
    Parameters:
    asset_value (float): Value of the firm's assets
    debt_value (float): Face value of debt
    risk_free_rate (float): Risk-free rate
    volatility (float): Volatility of the asset's value
    time_to_maturity (float): Time to maturity in years

    Returns:
    tuple: (Market value of equity, Market value of debt)
    """
    d1 = (np.log(asset_value / debt_value) + (risk_free_rate + 0.5 * volatility**2) * time_to_maturity) / (volatility * np.sqrt(time_to_maturity))
    d2 = d1 - volatility * np.sqrt(time_to_maturity)
    
    N_d1 = norm.cdf(d1)
    N_d2 = norm.cdf(d2)
    
    equity_value = asset_value * N_d1 - debt_value * np.exp(-risk_free_rate * time_to_maturity) * N_d2
    debt_value = asset_value - equity_value
    
    return equity_value, debt_value

In [ ]:
asset_value = 100
debt_value = 50
risk_free_rate = 0.05
volatility = 0.2
time_to_maturity = 1

equity_value, debt_value = black_scholes_valuation(asset_value, debt_value, risk_free_rate, volatility, time_to_maturity)
print(f"Market value of equity: {equity_value:.2f}")
print(f"Market value of debt: {debt_value:.2f}")
